In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import sys
sys.path.insert(0, '../')
from lib.model.unet_3d import UNet3D
from lib.model.unet_2d import UNet2D
from lib.model.corrector import Corrector
from lib.config.cfg import cfg
cfg['device'] = torch.device('cpu') #torch.device("cuda" if torch.cuda.is_available() else "cpu")


C:\Users\Viktor\ml\GlorysCorrection\notebooks\..\lib\config


In [2]:
bs, sl, c, z, h, w = 6, 4, 3, 34, 181, 577 
i2i = torch.ones([bs, z, h, w])
s2s = torch.ones([sl, bs, z, h, w])

In [3]:
unet2d = UNet2D(*cfg.model_args.unet2d.values())
model2d = Corrector(unet2d).to(cfg.device)
unet3d = UNet3D(*cfg.model_args.unet3d.values())
model3d = Corrector(unet3d).to(cfg.device)

In [4]:
model3d(torch.unsqueeze(i2i, 1)).shape

RuntimeError: [enforce fail at alloc_cpu.cpp:114] data. DefaultCPUAllocator: not enough memory: you tried to allocate 5454117888 bytes.

In [4]:
class Corrector(nn.Module):
    def __init__(self, model, channels=6):
        super().__init__()
        self.channels = channels
        self.unet = model

    def forward(self, x_orig):
        x = x_orig
        unet_out = self.unet(x)
        o_input = torch.split(x_orig, 3, dim=-3)
        return o_input[0] + unet_out.view(x.shape[0], x.shape[1], 3, x.shape[3], x.shape[4])

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool3d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose3d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CZHW
        diffZ = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        diffX = x2.size()[4] - x1.size()[4]
        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2,
                        diffZ // 2, diffZ - diffZ // 2])
        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)

In [6]:
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=False, factor=2):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.bilinear = bilinear

        self.inc = (DoubleConv(n_channels, 64 // factor))
        self.down1 = (Down(64 // factor, 128 // factor))
        self.down2 = (Down(128 // factor, 256 // factor))
        self.down3 = (Down(256 // factor, 512 // factor))
        bi_factor = 2 if bilinear else 1
        self.down4 = (Down(512 // factor, 1024 // factor // bi_factor))
        self.up1 = (Up(1024 // factor, 512 // factor // bi_factor, bilinear))
        self.up2 = (Up(512 // factor, 256 // factor // bi_factor, bilinear))
        self.up3 = (Up(256 // factor, 128 // factor // bi_factor, bilinear))
        self.up4 = (Up(128 // factor, 64 // factor, bilinear))
        self.outc = (OutConv(64 // factor, n_classes))

    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

In [7]:
unet_params = {"n_channels": 1,
               "n_classes": 1,
               "bilinear": True
               }
unet = UNet(*unet_params.values())
unet(torch.ones([6, 1, 34, 210, 280])).shape

torch.Size([6, 256, 2, 13, 17]) torch.Size([6, 256, 4, 26, 35]) 1
torch.Size([6, 256, 4, 26, 34]) torch.Size([6, 256, 4, 26, 35]) 2
0 0 1 diffZ diffY, diffX
torch.Size([6, 256, 4, 26, 35]) torch.Size([6, 256, 4, 26, 35]) 3
torch.Size([6, 128, 4, 26, 35]) torch.Size([6, 128, 8, 52, 70]) 1
torch.Size([6, 128, 8, 52, 70]) torch.Size([6, 128, 8, 52, 70]) 2
0 0 0 diffZ diffY, diffX
torch.Size([6, 128, 8, 52, 70]) torch.Size([6, 128, 8, 52, 70]) 3
torch.Size([6, 64, 8, 52, 70]) torch.Size([6, 64, 17, 105, 140]) 1
torch.Size([6, 64, 16, 104, 140]) torch.Size([6, 64, 17, 105, 140]) 2
1 1 0 diffZ diffY, diffX
torch.Size([6, 64, 17, 105, 140]) torch.Size([6, 64, 17, 105, 140]) 3
torch.Size([6, 32, 17, 105, 140]) torch.Size([6, 32, 34, 210, 280]) 1
torch.Size([6, 32, 34, 210, 280]) torch.Size([6, 32, 34, 210, 280]) 2
0 0 0 diffZ diffY, diffX
torch.Size([6, 32, 34, 210, 280]) torch.Size([6, 32, 34, 210, 280]) 3


torch.Size([6, 1, 34, 210, 280])

In [8]:
unet(i2i).shape

NameError: name 'i2i' is not defined

In [ ]:
stop

In [ ]:
import torch
from torch import nn
from transformers.models.bert.modeling_bert import BertEncoder
from transformers.models.bert.configuration_bert import BertConfig

class UNet(nn.Module):
    def __init__(self, n_channels, n_classes, bilinear=True):
        super(UNet, self).__init__()
        self.n_channels = n_channels
        self.n_classes = n_classes

        chan_factor = 2
        factor = 2 if bilinear else 1

#         self.patch_encoding = nn.Embedding(13 * 17, 512 // 2)
        self.patch_encoding = nn.Embedding(396, 512 // 2)
        self.pos_encoding = nn.Embedding(10, 512 // 2)

        self.causal_bro = BertEncoder(BertConfig(
            hidden_size=512 // 2,
            num_hidden_layers=4,
            num_attention_heads=4,
            intermediate_size=512 * 4 // 2
        ))

        self.inc = (DoubleConv(n_channels, 64 // chan_factor))
        self.down1 = (Down(64 // chan_factor, 128 // chan_factor))
        self.down2 = (Down(128 // chan_factor, 256 // chan_factor))
        self.down3 = (Down(256 // chan_factor, 512 // chan_factor))
        self.down4 = (Down(512 // chan_factor, 1024 // chan_factor // factor))
        self.up1 = (Up(1024 // chan_factor, 512 // chan_factor // factor, bilinear))
        self.up2 = (Up(512 // chan_factor, 256 // chan_factor // factor, bilinear))
        self.up3 = (Up(256 // chan_factor, 128 // chan_factor // factor, bilinear))
        self.up4 = (Up(128 // chan_factor, 64 // chan_factor))
        self.outc = (OutConv(64 // chan_factor, n_classes))

    def forward(self, x):
        orig_shape = x.shape
        x = x.view(-1, x.shape[-3], x.shape[-2], x.shape[-1])
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        # x3 = x3 + self.spatial_map_encode
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        emb_size = torch.mul(*x5.shape[-2:])
        emb_shape = x5.shape[-2:]
        print(x.shape, x1.shape, x2.shape,x3.shape,x4.shape,x5.shape,)
#         x5 = x5.view(orig_shape[0], orig_shape[1], 512 // 2, 13 * 17)
        x5 = x5.view(orig_shape[0], orig_shape[1], 512 // 2, emb_size)

        x5 = x5.permute(1, 0, 3, 2)
        print(x5.shape)
        x5_patch = self.patch_encoding(torch.arange(0, x5.shape[2], device=x5.device))
        x5_pos = self.pos_encoding(torch.arange(0, x5.shape[1], device=x5.device))
        print(x5_patch.shape, x5_pos.shape)
        x5 = x5 + x5_patch.view(1, 1, emb_size, 512 // 2) + x5_pos.view(1, orig_shape[0], 1, 512 // 2)
        x5 = x5.reshape(x5.shape[0], -1, 512 // 2)
        x5 = self.causal_bro(x5).last_hidden_state
        x5 = x5.reshape(x5.shape[0] * orig_shape[0], *emb_shape, 512 // 2)
        x5 = x5.permute(0, 3, 1, 2)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
#         x = x + self.spatial_map_decode
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits
        return x5


In [ ]:
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, mid_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(mid_channels, out_channels, kernel_size=3, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


class Down(nn.Module):
    """Downscaling with maxpool then double conv"""

    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)


class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, bilinear=True):
        super().__init__()

        # if bilinear, use the normal convolutions to reduce the number of channels
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
            self.conv = DoubleConv(in_channels, out_channels)
        else:
            self.up = nn.ConvTranspose2d(in_channels, in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        # input is CHW
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])

        # if you have padding issues, see
        # https://github.com/HaiyongJiang/U-Net-Pytorch-Unstructured-Buggy/commit/0e854509c2cea854e247a9c615f175f76fbb2e3a
        # https://github.com/xiaopeng-liao/Pytorch-UNet/commit/8ebac70e633bac59fc22bb5195e513d5832fb3bd
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)


class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)

    def forward(self, x):
        return self.conv(x)